<a href="https://colab.research.google.com/github/LorenzoCorbinelli/MLSA-project/blob/chunking/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [2]:
import os
import pandas as pd
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaConfig, AutoModel, DataCollatorForLanguageModeling, RobertaForMaskedLM
from datasets import Dataset as ds
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import Adam

# Load the dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Download the dataset
!wget http://files.srl.inf.ethz.ch/data/py150_files.tar.gz

Mounted at /content/drive
--2025-02-08 21:05:42--  http://files.srl.inf.ethz.ch/data/py150_files.tar.gz
Resolving files.srl.inf.ethz.ch (files.srl.inf.ethz.ch)... 129.132.114.90
Connecting to files.srl.inf.ethz.ch (files.srl.inf.ethz.ch)|129.132.114.90|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.sri.inf.ethz.ch/data/py150_files.tar.gz [following]
--2025-02-08 21:05:43--  https://files.sri.inf.ethz.ch/data/py150_files.tar.gz
Resolving files.sri.inf.ethz.ch (files.sri.inf.ethz.ch)... 129.132.114.90
Connecting to files.sri.inf.ethz.ch (files.sri.inf.ethz.ch)|129.132.114.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199067128 (190M) [application/gzip]
Saving to: ‘py150_files.tar.gz’

py150_files.tar.gz  100%[===================>] 189.84M  22.3MB/s    in 9.8s    

2025-02-08 21:05:54 (19.3 MB/s) - ‘py150_files.tar.gz’ saved [199067128/199067128]



In [5]:
!tar -xvzf py150_files.tar.gz # unzip the folder containing the dataset

data.tar.gz
github_repos.txt
python100k_train.txt
python50k_eval.txt
README.md


In [6]:
!tar -xvzf data.tar.gz # unzip the dataset

Output streaming troncato alle ultime 5000 righe.
data/mathcamp/pypicloud/git_hooks/hook.py
data/mathcamp/pypicloud/tests/
data/mathcamp/pypicloud/tests/test_simple.py
data/mathcamp/pypicloud/tests/test_packages.py
data/mathcamp/pypicloud/tests/test_access_backends.py
data/mathcamp/pypicloud/tests/test_security.py
data/mathcamp/pypicloud/tests/test_api.py
data/mathcamp/pypicloud/tests/test_storage.py
data/mathcamp/pypicloud/tests/test_util.py
data/mathcamp/pypicloud/tests/test_auth.py
data/mathcamp/pypicloud/pypicloud/
data/mathcamp/pypicloud/pypicloud/scripts.py
data/mathcamp/pypicloud/pypicloud/storage/
data/mathcamp/pypicloud/pypicloud/storage/files.py
data/mathcamp/pypicloud/pypicloud/views/
data/mathcamp/pypicloud/pypicloud/views/packages.py
data/mathcamp/pypicloud/pypicloud/views/simple.py
data/mathcamp/pypicloud/pypicloud/views/api.py
data/mathcamp/pypicloud/pypicloud/models.py
data/mathcamp/pypicloud/pypicloud/cache/
data/mathcamp/pypicloud/pypicloud/cache/__init__.py
data/math

After unzipping the data.tar.gz folder, we retrieve the python files, removing all the comments because they are not usefull for our purposes

Extract all the source code of the python files into a dataframe. One file is reported into a single line, including the reference of the file itself

In [7]:
def load_python_files(root_dir):
    """Loads all Python files in a directory recursively into a DataFrame."""
    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".py"):  # take only python files
                filepath = os.path.join(dirpath, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        content = f.readlines() # read the file line by line
                        filtered_content = []
                        inside_triple_quotes = False
                        for line in content:
                            stripped_line = line.strip()
                            if '"""' in stripped_line:
                                if stripped_line.count('"""') == 2:
                                    continue  # Ignore lines with both opening and closing triple quotes
                                inside_triple_quotes = not inside_triple_quotes
                                continue
                            if inside_triple_quotes or stripped_line.startswith('#'): # ignore line that starts with # or check if I am inside a multiline comment
                                continue
                            filtered_content.append(line)
                        all_files.append({'filepath': filepath, 'snippet_of_code': ''.join(filtered_content)})
                except Exception as e:
                    print(f"Error reading file {filepath}: {e}")
    return pd.DataFrame(all_files)


In [8]:
data_dir = "/content/data"
df_python_files = load_python_files(data_dir)

Error reading file /content/data/deanhiller/databus/webapp/play1.3.x/python/Lib/msilib/__init__.py: 'utf-8' codec can't decode byte 0xf6 in position 62: invalid start byte
Error reading file /content/data/kleientertainment/ds_mod_tools/pkg/win32/Python27/Lib/site-packages/PIL/WalImageFile.py: 'utf-8' codec can't decode byte 0xe4 in position 1876: invalid continuation byte
Error reading file /content/data/jpm/papercut/storage/forwarding_proxy.py: 'utf-8' codec can't decode byte 0xe4 in position 428: invalid continuation byte
Error reading file /content/data/hydroshare/hydroshare2/hs_docker_base/pysqlite-2.6.3/build/lib.macosx-10.9-intel-2.7/pysqlite2/dbapi2.py: 'utf-8' codec can't decode byte 0xe4 in position 113: invalid continuation byte
Error reading file /content/data/hydroshare/hydroshare2/hs_docker_base/pysqlite-2.6.3/build/lib.macosx-10.9-intel-2.7/pysqlite2/test/userfunctions.py: 'utf-8' codec can't decode byte 0xe4 in position 184: invalid continuation byte
Error reading file /

In [9]:
df_train = df_python_files["snippet_of_code"].iloc[0:1000] # take only the first 2000 snippets for training
df_eval = df_python_files["snippet_of_code"].iloc[1000:1500] # take 1000 snippets for evaluation

# Tokenization

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')
# tokenizer arguments to properly handle the tokenization of the snippets
tokenizer_kwargs = dict(truncation=True, padding=True, max_length=500, add_special_tokens=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [11]:
'''
We have used DataCollatorForLanguageModeling in order to tokenize the dataset and mask some tokens
It will automatically generate the labels for the masked tokens.
For the tokens not masked the label will be -100
'''
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [12]:
def tokenize_dataset(dataset, tokenizer, **kwargs):
    token_ids = tokenizer(dataset, return_tensors='pt', **kwargs)
    return token_ids

In [13]:
import pandas as pd

# Assuming df_train is a Series and tokenizer is already defined
def split_text_into_chunks(text, max_length=500):
    # Tokenize the full text first (without truncating)
    tokens = tokenizer.encode(text, add_special_tokens=True)

    # Split into chunks of max_length
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return chunks

def get_token_length(text):
    # Tokenize the text and return the number of tokens
    return len(tokenizer.encode(text, add_special_tokens=True))

# Create an empty list to collect new snippets
new_snippets = []

# Iterate over the Series (df_train)
for text in df_train:
    # Check the token length
    token_length = get_token_length(text)

    if token_length > 500:
        # Split the text into chunks if it's too long
        chunks = split_text_into_chunks(text)

        # Add each chunk as a new entry in the new_snippets list
        for chunk in chunks:
            # Decode the chunk back into text
            chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
            new_snippets.append(chunk_text)
    else:
        # If the text is small enough, keep the original snippet
        new_snippets.append(text)

# Create a new Series with the updated snippets
df_train_updated = pd.Series(new_snippets)

# Show the updated Series
df_train = df_train_updated

Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


In [23]:
df_train.size

4688

In [14]:
train_tensor_dataset = tokenize_dataset(df_train.to_list(), tokenizer, **tokenizer_kwargs)
eval_tensor_dataset = tokenize_dataset(df_eval.to_list(), tokenizer, **tokenizer_kwargs)

datasetTrain = ds.from_dict(train_tensor_dataset)
datasetEval = ds.from_dict(eval_tensor_dataset)

datasetTrain.set_format(type='torch', columns=['input_ids', 'attention_mask'])
datasetEval.set_format(type='torch', columns=['input_ids', 'attention_mask'])

generator = torch.Generator()
train_loader = DataLoader(datasetTrain, batch_size=4, shuffle=True, generator=generator, collate_fn=data_collator)
eval_loader = DataLoader(datasetEval, batch_size=4, collate_fn=data_collator)

In [15]:
model = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base-mlm')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(2):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key in tokenizer.model_input_names}
        inputs['labels'] = batch['labels'].to(device)
        outputs = model(**inputs)
        loss = outputs.loss
        print(loss)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} completed")


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

tensor(1.2685, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9493, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7775, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6883, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6836, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5528, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8209, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9246, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4139, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5823, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6119, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7482, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7243, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3718, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5211, device='cuda:0', grad_fn=

In [16]:
# Evaluation loop
'''
model.eval()
with torch.no_grad():
    for batch in eval_loader:
        input = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**input)
        print(outputs.loss)'''

'\nmodel.eval()\nwith torch.no_grad():\n    for batch in eval_loader:\n        input = {key: val.to(device) for key, val in batch.items()}\n        outputs = model(**input)\n        print(outputs.loss)'

In [26]:
# save the model

directory = '/content/drive/MyDrive/MLSAModelChuncked/'

model.save_pretrained(directory)
tokenizer.save_pretrained(directory)

('/content/drive/MyDrive/MLSAModelChuncked/tokenizer_config.json',
 '/content/drive/MyDrive/MLSAModelChuncked/special_tokens_map.json',
 '/content/drive/MyDrive/MLSAModelChuncked/vocab.json',
 '/content/drive/MyDrive/MLSAModelChuncked/merges.txt',
 '/content/drive/MyDrive/MLSAModelChuncked/added_tokens.json')

In [18]:
from tabulate import tabulate
def print_result(outputs):
    table_data = []
    for output in outputs:
        token_str = f'"{output["token_str"]}"'  # Preserve leading spaces by wrapping in quotes
        table_data.append([output['sequence'], token_str, output['score']])

    print("The suggested code completions are:")
    print(tabulate(table_data, headers=["Completion", "Predicted token", "Score"], tablefmt="grid", colalign=("left", "left", "center")) )

In [19]:
from transformers import pipeline
def code_completion(code_example, iterations: int = 1):
    '''
    - code_example: snipped of code that need to be code-completed. No token <mask> needed.
    - iterations: number of subsequent code completions to be generated.
                  Each sequence generated after the first one will be based only on the previous sequence with the highest score.
    '''
    code_example = code_example + "<mask>"
    fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
    current_example = code_example  # Start with the initial code

    for _ in range(iterations):
        outputs = fill_mask(current_example)

        # Take the first prediction and append <mask> to continue completion
        best_prediction = outputs[0]["sequence"]
        current_example = best_prediction + " <mask>"
        print_result(outputs)
    return outputs

In [20]:
result = code_completion("def is_zero(x): return x==")

Device set to use cuda:0


The suggested code completions are:
+---------------------------------+-------------------+-----------+
| Completion                      | Predicted token   |   Score   |
+=================================+===================+===========+
| def is_zero(x): return x== 0    | " 0"              |  0.4718   |
+---------------------------------+-------------------+-----------+
| def is_zero(x): return x==0     | "0"               | 0.342096  |
+---------------------------------+-------------------+-----------+
| def is_zero(x): return x== 1    | " 1"              | 0.026647  |
+---------------------------------+-------------------+-----------+
| def is_zero(x): return x== zero | " zero"           | 0.0167545 |
+---------------------------------+-------------------+-----------+
| def is_zero(x): return x==x     | "x"               | 0.0152117 |
+---------------------------------+-------------------+-----------+


In [21]:
result = code_completion("def add(a, b): return a", 2)

Device set to use cuda:0


The suggested code completions are:
+---------------------------+-------------------+-----------+
| Completion                | Predicted token   |   Score   |
+===========================+===================+===========+
| def add(a, b): return a + | " +"              | 0.248686  |
+---------------------------+-------------------+-----------+
| def add(a, b): return a - | " -"              | 0.139644  |
+---------------------------+-------------------+-----------+
| def add(a, b): return a b | " b"              | 0.134556  |
+---------------------------+-------------------+-----------+
| def add(a, b): return a   | "                 | 0.109213  |
|                           | "                 |           |
+---------------------------+-------------------+-----------+
| def add(a, b): return a   | " "               | 0.0376533 |
+---------------------------+-------------------+-----------+
The suggested code completions are:
+-----------------------------+-------------------+---------

In [22]:
result = code_completion("for element ", 2)

Device set to use cuda:0


The suggested code completions are:
+------------------+-------------------+-----------+
| Completion       | Predicted token   |   Score   |
+==================+===================+===========+
| for element.     | "."               | 0.119692  |
+------------------+-------------------+-----------+
| for element in   | " in"             | 0.109932  |
+------------------+-------------------+-----------+
| for elementname  | "name"            | 0.0423685 |
+------------------+-------------------+-----------+
| for elementlist  | "list"            | 0.0383944 |
+------------------+-------------------+-----------+
| for element list | " list"           | 0.0354086 |
+------------------+-------------------+-----------+
The suggested code completions are:
+---------------------+-------------------+-----------+
| Completion          | Predicted token   |   Score   |
+=====================+===================+===========+
| for element.html    | "html"            | 0.0684438 |
+--------------

In [55]:
result = code_completion('numbers = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]\nevens = [n for n in numbers if n %', 2)

Device set to use cuda:0


The suggested code completions are:
+-------------------------------------------------+-------------------+-----------+
| Completion                                      | Predicted token   |   Score   |
+=================================================+===================+===========+
| numbers = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30] | " 2"              | 0.311784  |
| evens = [n for n in numbers if n % 2            |                   |           |
+-------------------------------------------------+-------------------+-----------+
| numbers = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30] | " 10"             | 0.176871  |
| evens = [n for n in numbers if n % 10           |                   |           |
+-------------------------------------------------+-------------------+-----------+
| numbers = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30] | " 3"              | 0.0768997 |
| evens = [n for n in numbers if n % 3            |                   |           |
+---------------------------------------

In [57]:
result = code_completion('import http.server, socketserver\nwith socketserver.TCPServer(('', 8000), http.server.SimpleHTTPRequestHandler) as httpd: httpd', 4)

Device set to use cuda:0


The suggested code completions are:
+-----------------------------------------------------------------------------------------------+-------------------+-----------+
| Completion                                                                                    | Predicted token   |   Score   |
+===============================================================================================+===================+===========+
| import http.server, socketserver                                                              | "."               | 0.440149  |
| with socketserver.TCPServer((, 8000), http.server.SimpleHTTPRequestHandler) as httpd: httpd.  |                   |           |
+-----------------------------------------------------------------------------------------------+-------------------+-----------+
| import http.server, socketserver                                                              | " ="              | 0.302568  |
| with socketserver.TCPServer((, 8000), http.server.Si

In [58]:
result = code_completion('class Circle: def __init__(self, r): self._r = r\n@property\ndef area(self): return 3.14 * self._r ** 2\nc = Circle(5)\na', 3)

Device set to use cuda:0


The suggested code completions are:
+--------------------------------------------------+-------------------+-----------+
| Completion                                       | Predicted token   |   Score   |
+==================================================+===================+===========+
| class Circle: def __init__(self, r): self._r = r | " ="              | 0.352969  |
| @property                                        |                   |           |
| def area(self): return 3.14 * self._r ** 2       |                   |           |
| c = Circle(5)                                    |                   |           |
| a =                                              |                   |           |
+--------------------------------------------------+-------------------+-----------+
| class Circle: def __init__(self, r): self._r = r | "="               | 0.0383078 |
| @property                                        |                   |           |
| def area(self): return 3.14

In [60]:
result = code_completion('from dataclasses import dataclass\n@dataclass\nclass Car: make: str; model: str; year: int\nc = ',3)

Device set to use cuda:0


The suggested code completions are:
+---------------------------------------------+-------------------+-----------+
| Completion                                  | Predicted token   |   Score   |
+=============================================+===================+===========+
| from dataclasses import dataclass           | " Car"            | 0.236533  |
| @dataclass                                  |                   |           |
| class Car: make: str; model: str; year: int |                   |           |
| c = Car                                     |                   |           |
+---------------------------------------------+-------------------+-----------+
| from dataclasses import dataclass           | " {"              | 0.123479  |
| @dataclass                                  |                   |           |
| class Car: make: str; model: str; year: int |                   |           |
| c = {                                       |                   |           |
+---

In [63]:
result = code_completion('import threading\ndef worker(): return sum(range(100000))\nth = threading.Thread(target=worker)\nth.start()\nth',3)

Device set to use cuda:0


The suggested code completions are:
+-----------------------------------------+-------------------+-----------+
| Completion                              | Predicted token   |   Score   |
+=========================================+===================+===========+
| import threading                        | "."               | 0.355974  |
| def worker(): return sum(range(100000)) |                   |           |
| th = threading.Thread(target=worker)    |                   |           |
| th.start()                              |                   |           |
| th.                                     |                   |           |
+-----------------------------------------+-------------------+-----------+
| import threading                        | "()"              | 0.175539  |
| def worker(): return sum(range(100000)) |                   |           |
| th = threading.Thread(target=worker)    |                   |           |
| th.start()                              |         

In [74]:
result = code_completion('numbers = [1, 2, 3, 4, 5]\nsquares = [n**2 for',4)

Device set to use cuda:0


The suggested code completions are:
+---------------------------+-------------------+-----------+
| Completion                | Predicted token   |   Score   |
+===========================+===================+===========+
| numbers = [1, 2, 3, 4, 5] | " n"              | 0.467954  |
| squares = [n**2 for n     |                   |           |
+---------------------------+-------------------+-----------+
| numbers = [1, 2, 3, 4, 5] | " x"              | 0.031409  |
| squares = [n**2 for x     |                   |           |
+---------------------------+-------------------+-----------+
| numbers = [1, 2, 3, 4, 5] | " p"              | 0.0242665 |
| squares = [n**2 for p     |                   |           |
+---------------------------+-------------------+-----------+
| numbers = [1, 2, 3, 4, 5] | " i"              | 0.019855  |
| squares = [n**2 for i     |                   |           |
+---------------------------+-------------------+-----------+
| numbers = [1, 2, 3, 4, 5] | " N"

In [86]:
result = code_completion('numbers = [1, 2, 3, 4, 5, 6, 7]\nodd_numbers = [n for n in numbers if',4)

Device set to use cuda:0


The suggested code completions are:
+-------------------------------------------+-------------------+------------+
| Completion                                | Predicted token   |   Score    |
+===========================================+===================+============+
| numbers = [1, 2, 3, 4, 5, 6, 7]           | " n"              |  0.899745  |
| odd_numbers = [n for n in numbers if n    |                   |            |
+-------------------------------------------+-------------------+------------+
| numbers = [1, 2, 3, 4, 5, 6, 7]           | " not"            | 0.0230192  |
| odd_numbers = [n for n in numbers if not  |                   |            |
+-------------------------------------------+-------------------+------------+
| numbers = [1, 2, 3, 4, 5, 6, 7]           | " 1"              | 0.0109042  |
| odd_numbers = [n for n in numbers if 1    |                   |            |
+-------------------------------------------+-------------------+------------+
| numbers = [1, 

In [97]:
result = code_completion('list1 = [1, 2, 3]\nlist2 = [4, 5, 6]\nresult = list1 ',3)

Device set to use cuda:0


The suggested code completions are:
+---------------------+-------------------+-----------+
| Completion          | Predicted token   |   Score   |
+=====================+===================+===========+
| list1 = [1, 2, 3]   | " +"              | 0.315822  |
| list2 = [4, 5, 6]   |                   |           |
| result = list1 +    |                   |           |
+---------------------+-------------------+-----------+
| list1 = [1, 2, 3]   | "                 | 0.0871726 |
| list2 = [4, 5, 6]   | "                 |           |
| result = list1      |                   |           |
+---------------------+-------------------+-----------+
| list1 = [1, 2, 3]   | " list"           | 0.061932  |
| list2 = [4, 5, 6]   |                   |           |
| result = list1 list |                   |           |
+---------------------+-------------------+-----------+
| list1 = [1, 2, 3]   | ","               | 0.0475122 |
| list2 = [4, 5, 6]   |                   |           |
| result = l

In [101]:
result = code_completion("data = ['apple', 'banana', 'apple', 'cherry', 'apple']\ncount_apples = data.count('", 3)

Device set to use cuda:0


The suggested code completions are:
+--------------------------------------------------------+-------------------+-----------+
| Completion                                             | Predicted token   |   Score   |
+========================================================+===================+===========+
| data = ['apple', 'banana', 'apple', 'cherry', 'apple'] | "apple"           | 0.229796  |
| count_apples = data.count('apple                       |                   |           |
+--------------------------------------------------------+-------------------+-----------+
| data = ['apple', 'banana', 'apple', 'cherry', 'apple'] | "app"             | 0.0884307 |
| count_apples = data.count('app                         |                   |           |
+--------------------------------------------------------+-------------------+-----------+
| data = ['apple', 'banana', 'apple', 'cherry', 'apple'] | "apps"            | 0.0756386 |
| count_apples = data.count('apps                     